# 讀取文件



In [2]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv(r"C:\Users\USER\Downloads\DATAMINING\test.csv")
sample_submission_df = pd.read_csv(r"C:\Users\USER\Downloads\DATAMINING\sample_submission.csv")
print("ok")

ok


In [ ]:
#資料處理

In [188]:
#去掉不需要的columns
parts_train_df = train_df.copy()
parts_train_df['Fitness_Endurance-Time_Mins'] = train_df['Fitness_Endurance-Time_Mins'] * 60 + train_df['Fitness_Endurance-Time_Mins']
columns_to_drop = [
    'PCIAT-Season', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03',
    'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07',
    'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11',
    'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15',
    'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19',
    'PCIAT-PCIAT_20', 'PCIAT-PCIAT_Total','Fitness_Endurance-Time_Mins',
    'Physical-Height','Physical-Weight','Physical-Waist_Circumference',
    'BIA-BIA_BMI','BIA-BIA_ECW','BIA-BIA_ICW','BIA-BIA_FFM'
]
parts_train_df = parts_train_df.drop(columns=columns_to_drop)
print("已經刪除欄位")

已經刪除欄位


In [189]:
#增加需要的columns
Waist_Circumference_notnull = train_df['Physical-Waist_Circumference'].notnull()
parts_train_df.loc[Waist_Circumference_notnull, 'Height_Dividing_Waist_Circumference'] = (
    train_df.loc[Waist_Circumference_notnull, 'Physical-Height'] /
    train_df.loc[Waist_Circumference_notnull, 'Physical-Waist_Circumference']
)

BIA_BIA_ECW_notnull = train_df['BIA-BIA_ECW'].notnull()
parts_train_df.loc[BIA_BIA_ECW_notnull, 'BIA-BIA_ECW_dividing_ICW'] = (
    train_df.loc[BIA_BIA_ECW_notnull, 'BIA-BIA_ECW'] /
    train_df.loc[BIA_BIA_ECW_notnull, 'BIA-BIA_ICW']
)

print("已經增加欄位")

已經增加欄位


In [ ]:
#將parts_train_df轉換成離散的型態存到Discretization_parts_train_df

In [279]:
Discretization_parts_train_df=parts_train_df.copy()
#season轉換
season_mapping = {
    'Spring': 1,
    'Summer': 2,
    'Fall': 3,
    'Winter': 4
}
columns_to_convert = ['FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season','Basic_Demos-Enroll_Season',
                      'CGAS-Season', 'Physical-Season','Fitness_Endurance-Season','PreInt_EduHx-Season','SDS-Season']
for col in columns_to_convert:
    Discretization_parts_train_df[col] = parts_train_df[col].map(season_mapping)
    Discretization_parts_train_df[col] = Discretization_parts_train_df[col].fillna(0).astype(int)



#Height_Dividing_Waist_Circumference轉換
Discretization_parts_train_df['Height_Dividing_Waist_Circumference'] = parts_train_df['Height_Dividing_Waist_Circumference'].apply(
    lambda x: 1 if x >= 2 else (0 if x < 2 else -1)
)



#CGAS_Level轉換
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Discretization_parts_train_df['CGAS_Level'] = pd.cut(Discretization_parts_train_df['CGAS-CGAS_Score'], bins=bins, labels=labels, right=True)
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['CGAS-CGAS_Score'])



# BMI轉換
# 男孩的 BMI 評級範圍
bmi_ranges_male = {
    5: {"1": 13.5, "2": 14.5, "3": 15.5, "4": 17, "5": float('inf')},
    6: {"1": 14, "2": 15, "3": 16, "4": 18, "5": float('inf')},
    7: {"1": 14.5, "2": 15.5, "3": 17, "4": 19, "5": float('inf')},
    8: {"1": 15, "2": 16, "3": 18, "4": 20, "5": float('inf')},
    9: {"1": 15.5, "2": 16.5, "3": 19, "4": 21, "5": float('inf')},
    10: {"1": 16, "2": 17, "3": 20, "4": 22, "5": float('inf')},
    11: {"1": 16.5, "2": 17.5, "3": 21, "4": 23, "5": float('inf')},
    12: {"1": 17, "2": 18, "3": 22, "4": 24, "5": float('inf')},
    13: {"1": 17.5, "2": 18.5, "3": 23, "4": 25, "5": float('inf')},
    14: {"1": 18, "2": 19, "3": 24, "4": 26, "5": float('inf')},
    15: {"1": 18.5, "2": 19.5, "3": 25, "4": 27, "5": float('inf')},
    16: {"1": 19, "2": 20, "3": 26, "4": 28, "5": float('inf')},
    17: {"1": 19.5, "2": 20.5, "3": 27, "4": 29, "5": float('inf')},
    18: {"1": 20, "2": 21, "3": 28, "4": 30, "5": float('inf')},
    19: {"1": 20.5, "2": 21.5, "3": 29, "4": 31, "5": float('inf')}
}
# 女孩的 BMI 評級範圍
bmi_ranges_female = {
    5: {"1": 13, "2": 14, "3": 15, "4": 16.5, "5": float('inf')},
    6: {"1": 13.5, "2": 14.5, "3": 15.5, "4": 17, "5": float('inf')},
    7: {"1": 14, "2": 15, "3": 16, "4": 18, "5": float('inf')},
    8: {"1": 14.5, "2": 15.5, "3": 17, "4": 19, "5": float('inf')},
    9: {"1": 15, "2": 16, "3": 18, "4": 20, "5": float('inf')},
    10: {"1": 15.5, "2": 16.5, "3": 19, "4": 21, "5": float('inf')},
    11: {"1": 16, "2": 17, "3": 20, "4": 22, "5": float('inf')},
    12: {"1": 16.5, "2": 17.5, "3": 21, "4": 23, "5": float('inf')},
    13: {"1": 17, "2": 18, "3": 22, "4": 24, "5": float('inf')},
    14: {"1": 17.5, "2": 18.5, "3": 23, "4": 25, "5": float('inf')},
    15: {"1": 18, "2": 19, "3": 24, "4": 26, "5": float('inf')},
    16: {"1": 18.5, "2": 19.5, "3": 25, "4": 27, "5": float('inf')},
    17: {"1": 19, "2": 20, "3": 26, "4": 28, "5": float('inf')},
    18: {"1": 19.5, "2": 20.5, "3": 27, "4": 29, "5": float('inf')},
    19: {"1": 20, "2": 21, "3": 28, "4": 30, "5": float('inf')}
}
#分配 BMI 級別
def assign_bmi_category(age, bmi, sex):
    if sex == 0:  # 男性
        bmi_ranges = bmi_ranges_male
    elif sex == 1:  # 女性
        bmi_ranges = bmi_ranges_female
    else:
        return None 

    if age in bmi_ranges:
        for category, threshold in bmi_ranges[age].items():
            if bmi < threshold:
                return category
    return None  # 年齡超出範圍的情況

Discretization_parts_train_df['BMI_Category'] = train_df.apply(
    lambda row: assign_bmi_category(row['Basic_Demos-Age'], row['Physical-BMI'], row['Basic_Demos-Sex']), axis=1
)
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['Physical-BMI'])
Discretization_parts_train_df['BMI_Category'] = parts_train_df.apply(lambda x: calculate_bmi_category(x['Basic_Demos-Age'], x['Physical-BMI']), axis=1)
Discretization_parts_train_df['BMI_Category']=Discretization_parts_train_df['BMI_Category'].fillna(-1).astype(int)


#Physical-Diastolic_BP
Discretization_parts_train_df['Physical-Diastolic_BP']=Discretization_parts_train_df['Physical-Diastolic_BP'].fillna(-1).astype(int)


#Physical-HeartRate
Discretization_parts_train_df['Physical-HeartRate']=Discretization_parts_train_df['Physical-HeartRate'].fillna(-1).astype(int)


#Physical-Systolic_BP
Discretization_parts_train_df['Physical-Systolic_BP']=Discretization_parts_train_df['Physical-Systolic_BP'].fillna(-1).astype(int)


#Fitness_Endurance-Max_Stage
Discretization_parts_train_df['Fitness_Endurance-Max_Stage']=Discretization_parts_train_df['Fitness_Endurance-Max_Stage'].fillna(-1).astype(int)


#Fitness_Endurance-Time_Sec
Discretization_parts_train_df['Fitness_Endurance-Time_Sec']=Discretization_parts_train_df['Fitness_Endurance-Time_Sec'].fillna(-1).astype(int)


#FGC-FGC_CU
Discretization_parts_train_df['FGC-FGC_CU']=Discretization_parts_train_df['FGC-FGC_CU'].fillna(-1).astype(int)


#FGC-FGC_CU_Zone
Discretization_parts_train_df['FGC-FGC_CU_Zone']=Discretization_parts_train_df['FGC-FGC_CU_Zone'].fillna(-1).astype(int)


#FGC-FGC_GSND
Discretization_parts_train_df['FGC-FGC_GSND_tentimes'] = Discretization_parts_train_df['FGC-FGC_GSND'].fillna(-0.1) * 10
Discretization_parts_train_df['FGC-FGC_GSND_tentimes'] = Discretization_parts_train_df['FGC-FGC_GSND_tentimes'].astype(int)
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['FGC-FGC_GSND'])


#FGC-FGC_GSND_Zone
Discretization_parts_train_df['FGC-FGC_GSND_Zone']=Discretization_parts_train_df['FGC-FGC_GSND_Zone'].fillna(-1).astype(int)


#FGC-FGC_GSD
Discretization_parts_train_df['FGC-FGC_GSD_tentimes'] = Discretization_parts_train_df['FGC-FGC_GSD'].fillna(-0.1) * 10
Discretization_parts_train_df['FGC-FGC_GSD_tentimes'] = Discretization_parts_train_df['FGC-FGC_GSD_tentimes'].astype(int)
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['FGC-FGC_GSD'])


#FGC-FGC_GSD_Zone
Discretization_parts_train_df['FGC-FGC_GSD_Zone']=Discretization_parts_train_df['FGC-FGC_GSD_Zone'].fillna(-1).astype(int)


#FGC-FGC_PU
Discretization_parts_train_df['FGC-FGC_PU']=Discretization_parts_train_df['FGC-FGC_PU'].fillna(-1).astype(int)


#FGC-FGC_PU
Discretization_parts_train_df['FGC-FGC_PU_Zone']=Discretization_parts_train_df['FGC-FGC_PU_Zone'].fillna(-1).astype(int)


#FGC-FGC_SRL
Discretization_parts_train_df['FGC-FGC_SRL']=Discretization_parts_train_df['FGC-FGC_SRL'].fillna(-1).astype(int)


#FGC-FGC_SRL_Zone
Discretization_parts_train_df['FGC-FGC_SRL_Zone']=Discretization_parts_train_df['FGC-FGC_SRL_Zone'].fillna(-1).astype(int)


#FGC-FGC_SRL_Zone
Discretization_parts_train_df['FGC-FGC_SRR']=Discretization_parts_train_df['FGC-FGC_SRR'].fillna(-1).astype(int)


#FGC-FGC_SRR_Zone
Discretization_parts_train_df['FGC-FGC_SRR_Zone']=Discretization_parts_train_df['FGC-FGC_SRR_Zone'].fillna(-1).astype(int)


#FGC-FGC_TL
Discretization_parts_train_df['FGC-FGC_TL']=Discretization_parts_train_df['FGC-FGC_TL'].fillna(-1).astype(int)


#FGC-FGC_TL_Zone
Discretization_parts_train_df['FGC-FGC_TL_Zone']=Discretization_parts_train_df['FGC-FGC_TL_Zone'].fillna(-1).astype(int)


#BIA-BIA_Activity_Level_num
Discretization_parts_train_df['BIA-BIA_Activity_Level_num']=Discretization_parts_train_df['BIA-BIA_Activity_Level_num'].fillna(-1).astype(int)


#BIA-BIA_ECW_dividing_ICW
Discretization_parts_train_df['BIA-BIA_ECW_dividing_ICW_Level'] = parts_train_df['BIA-BIA_ECW_dividing_ICW'].apply(lambda x: 2 if x >= 0.39 else (0 if x < 0.36 else (1 if x>0 else -1)))
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['BIA-BIA_ECW_dividing_ICW'])


#BIA-BIA_FFMI
Discretization_parts_train_df['BIA-BIA_FFMI_thousandstimes'] = Discretization_parts_train_df['BIA-BIA_FFMI'].fillna(-0.001) * 1000
Discretization_parts_train_df['BIA-BIA_FFMI_thousandstimes'] = Discretization_parts_train_df['BIA-BIA_FFMI_thousandstimes'].astype(int)
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['BIA-BIA_FFMI'])


#FMI
Discretization_parts_train_df['BIA-BIA_FMI_tenthousandstimes'] = Discretization_parts_train_df['BIA-BIA_FMI'].fillna(-0.0001) * 10000
Discretization_parts_train_df['BIA-BIA_FMI_tenthousandstimes'] = Discretization_parts_train_df['BIA-BIA_FMI_tenthousandstimes'].astype(int)
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['BIA-BIA_FMI'])


#BIA-BIA_Fat
Discretization_parts_train_df['BIA-BIA_Fat'] = Discretization_parts_train_df['BIA-BIA_Fat'].apply(lambda x: -0.0001 if x < 0 else x)
Discretization_parts_train_df['BIA-BIA_Fat_tenthousandstimes'] = Discretization_parts_train_df['BIA-BIA_Fat'].fillna(-0.0001) * 10000
Discretization_parts_train_df['BIA-BIA_Fat_tenthousandstimes'] = Discretization_parts_train_df['BIA-BIA_Fat_tenthousandstimes'].astype(int)
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['BIA-BIA_Fat'])


#BIA-BIA_Frame_num
Discretization_parts_train_df['BIA-BIA_Frame_num']=Discretization_parts_train_df['BIA-BIA_Frame_num'].fillna(-1).astype(int)


#BIA-BIA_LDM
Discretization_parts_train_df['BIA-BIA_LDM_tenthousandstimes'] = Discretization_parts_train_df['BIA-BIA_LDM'].fillna(-0.0001) * 10000
Discretization_parts_train_df['BIA-BIA_LDM_tenthousandstimes'] = Discretization_parts_train_df['BIA-BIA_LDM_tenthousandstimes'].astype(int)
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['BIA-BIA_LDM'])


#PreInt_EduHx-computerinternet_hoursday
Discretization_parts_train_df['PreInt_EduHx-computerinternet_hoursday']=Discretization_parts_train_df['PreInt_EduHx-computerinternet_hoursday'].fillna(-1).astype(int)


#SDS-SDS_Total_T
Discretization_parts_train_df['SDS-SDS_Total_T']=Discretization_parts_train_df['SDS-SDS_Total_T'].fillna(-1).astype(int)


#SDS-SDS_Total_Raw
Discretization_parts_train_df['SDS-SDS_Total_Raw']=Discretization_parts_train_df['SDS-SDS_Total_Raw'].fillna(-1).astype(int)


#PAQ_C-PAQ_C_Total
Discretization_parts_train_df['PAQ_C-PAQ_C_Total_thousandstimes'] = Discretization_parts_train_df['PAQ_C-PAQ_C_Total'].fillna(-0.001) * 1000
Discretization_parts_train_df['PAQ_C-PAQ_C_Total_thousandstimes'] = Discretization_parts_train_df['PAQ_C-PAQ_C_Total_thousandstimes'].astype(int)
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['PAQ_C-PAQ_C_Total'])


#PAQ_A-PAQ_A_Total
Discretization_parts_train_df['PAQ_A-PAQ_A_Total_hundredstimes'] = Discretization_parts_train_df['PAQ_A-PAQ_A_Total'].fillna(-0.01) * 100
Discretization_parts_train_df['PAQ_A-PAQ_A_Total_hundredstimes'] = Discretization_parts_train_df['PAQ_A-PAQ_A_Total_hundredstimes'].astype(int)
Discretization_parts_train_df = Discretization_parts_train_df.drop(columns=['PAQ_A-PAQ_A_Total'])


#sii
Discretization_parts_train_df['sii']=Discretization_parts_train_df['sii'].fillna(-1).astype(int)
print("已經轉換為離散型態")

已經轉換為離散型態


In [287]:
# 選擇非整數類型的列
non_integer_columns = Discretization_parts_train_df.select_dtypes(exclude=['int']).columns

# 查看這些列
print("還要處理:")
print("Non-integer columns:")
print(non_integer_columns)

還要處理:
Non-integer columns:
Index(['id', 'BIA-BIA_BMC', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_LST',
       'BIA-BIA_SMM', 'BIA-BIA_TBW', 'CGAS_Level'],
      dtype='object')


In [282]:
Discretization_parts_train_df

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,Physical-Season,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,...,BMI_Category,FGC-FGC_GSND_tentimes,FGC-FGC_GSD_tentimes,BIA-BIA_ECW_dividing_ICW_Level,BIA-BIA_FFMI_thousandstimes,BIA-BIA_FMI_tenthousandstimes,BIA-BIA_Fat_tenthousandstimes,BIA-BIA_LDM_tenthousandstimes,PAQ_C-PAQ_C_Total_thousandstimes,PAQ_A-PAQ_A_Total_hundredstimes
0,00008ff9,3,5,0,4,3,-1,-1,-1,0,...,1,-1,-1,0,13817,30614,92137,88953,-1,-1
1,000fd460,2,9,0,0,3,75,70,122,0,...,-2,-1,-1,0,12825,12117,39708,149740,2340,-1
2,00105258,2,10,1,3,3,65,94,117,3,...,-1,102,147,-1,-1,-1,-1,-1,2170,-1
3,00115b9f,4,9,0,3,2,60,97,117,2,...,0,-1,-1,2,14074,42203,188243,167790,2451,-1
4,0016bb22,1,18,1,2,0,-1,-1,-1,0,...,2,-1,-1,-1,-1,-1,-1,-1,-1,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,3,13,0,1,3,71,70,104,0,...,-2,180,199,2,14062,23013,115883,179797,3260,-1
3956,ffa9794a,4,10,0,0,1,60,78,118,0,...,0,-1,-1,-1,-1,-1,-1,-1,2340,-1
3957,ffcd4dbd,3,11,0,1,4,79,99,116,0,...,1,185,158,2,14804,66395,339966,213403,2729,-1
3958,ffed1dd5,1,13,0,1,4,59,61,113,0,...,-2,-1,-1,2,13068,-8311,-1,250054,3300,-1


In [289]:
FPA_Discretization_parts_train_df=Discretization_parts_train_df.copy()
columns_to_drop = ['id', 'BIA-BIA_BMC', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_LST',
       'BIA-BIA_SMM', 'BIA-BIA_TBW', 'CGAS_Level']
FPA_Discretization_parts_train_df = FPA_Discretization_parts_train_df.drop(columns=columns_to_drop)
print("ok")

ok


In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# 假設 'FPA_Discretization_parts_train_df' 是你的原始 DataFrame
batch_size = 1000  # 每次處理的行數
frequent_itemsets_all = []  # 儲存所有批次的頻繁項目集

# 分批處理資料
for start in range(0, len(FPA_Discretization_parts_train_df), batch_size):
    end = start + batch_size
    batch_df = FPA_Discretization_parts_train_df.iloc[start:end]
    
    # 將數據轉為布林格式
    batch_bool_df = batch_df.applymap(lambda x: x > 0)
    
    # 使用 apriori 挖掘頻繁項目集，啟用 low_memory 以節省內存
    frequent_itemsets = apriori(batch_bool_df, min_support=0.1, use_colnames=True, low_memory=True)
    
    # 將該批次的結果添加到列表中
    frequent_itemsets_all.append(frequent_itemsets)

# 合併所有批次的頻繁項目集，並移除重複項目
all_frequent_itemsets = pd.concat(frequent_itemsets_all).drop_duplicates()

# 生成關聯規則
FPA_Discretization_parts_train_df_rules = association_rules(all_frequent_itemsets, metric="confidence", min_threshold=0.7)

# 顯示結果
print(all_frequent_itemsets)
print(FPA_Discretization_parts_train_df_rules)


C:\Users\USER\AppData\Local\Temp\ipykernel_23648\424034209.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  batch_bool_df = batch_df.applymap(lambda x: x > 0)
